In [19]:
import sys
!{sys.executable} -m pip install mysql-connector-python
import mysql.connector

In [20]:
import mysql.connector

# Functions to create database and tables inside database

def create_database(database_name):
    db = mysql.connector.connect(
        host = "localhost",
        user = "root",
        passwd = "password"
    )
    mycursor = db.cursor()
    mycursor.execute("CREATE DATABASE IF NOT EXISTS " + database_name)
    return

def create_tables(database_name):
    db = mysql.connector.connect(
        host = "localhost",
        user = "root",
        passwd = "password",
        database = database_name
    )
    mycursor = db.cursor()
    mycursor.execute("CREATE TABLE IF NOT EXISTS Schedule(room_no INT, date_ DATE, time_ INT, booking_status INT, PRIMARY KEY (room_no, date_, time_));")
    mycursor.execute("CREATE TABLE IF NOT EXISTS Weather(date_ DATE, time_ INT, outside_temp FLOAT, PRIMARY KEY (date_, time_));")
    mycursor.execute("CREATE TABLE IF NOT EXISTS Room(room_no INT, date_ DATE, time_ INT, inside_temp FLOAT, change_temp INT, PRIMARY KEY (room_no, date_, time_));")
    mycursor.execute("CREATE TABLE IF NOT EXISTS Energy(room_no INT, date_ DATE, time_ INT, energy_consumption FLOAT, PRIMARY KEY (room_no, date_));")
    return

create_database("testdata")
create_tables("testdata")

In [21]:
def set_database(database_name):
    db = mysql.connector.connect(
        host = "localhost",
        user = "root",
        passwd = "password",
        database = database_name
    )
    return db

db = set_database("testdata")
# Sets target database for following functions

In [22]:
# Insert and Get functions for tables

def insert_schedule_data(db, room_no,date_,time_,booking_status):
    mycursor = db.cursor()
    mycursor.execute("INSERT INTO Schedule (room_no, date_, time_, booking_status) VALUES(%s,%s,%s,%s)", ( room_no,date_,time_,booking_status))
    db.commit()
    mycursor.execute("SELECT * FROM Schedule")
    for x in mycursor: print(x)

def get_schedule_data(db,room_no,date_,time_):
    mycursor = db.cursor()
    mycursor.execute("SELECT booking_status FROM Schedule WHERE room_no=%s AND date_=%s AND time_=%s",(room_no,date_,time_))
    for x in mycursor: return x

def insert_weather_data(db, date_,time_,outside_temp):
    mycursor = db.cursor()
    mycursor.execute("INSERT INTO Weather (date_, time_, outside_temp) VALUES(%s,%s,%s)", (date_,time_,outside_temp))
    db.commit()
    mycursor.execute("SELECT * FROM Weather")
    for x in mycursor: print(x)

def get_weather_data(db,date_,time_):
    mycursor = db.cursor()
    mycursor.execute("SELECT outside_temp FROM Weather WHERE date_=%s AND time_=%s",(date_,time_))
    for x in mycursor: return x

def insert_room_data(db, room_no,date_,time_,inside_temp,change_temp):
    mycursor = db.cursor()
    mycursor.execute("INSERT INTO Room (room_no, date_, time_, inside_temp, change_temp) VALUES(%s,%s,%s,%s,%s)", ( room_no,date_,time_,inside_temp,change_temp))
    db.commit()
    mycursor.execute("SELECT * FROM Room")
    for x in mycursor: print(x)

def get_room_data(db,room_no,date_,time_):
    mycursor = db.cursor()
    mycursor.execute("SELECT inside_temp, change_temp FROM Room WHERE room_no=%s AND date_=%s AND time_=%s",(room_no,date_,time_))
    for x in mycursor: return x

def insert_energy_data(db, room_no,date_,time_,energy_consumption):
    mycursor = db.cursor()
    mycursor.execute("INSERT INTO Energy (room_no, date_, time_, energy_consumption) VALUES(%s,%s,%s,%s)", (room_no,date_,time_,energy_consumption))
    db.commit()
    mycursor.execute("SELECT * FROM Energy")
    for x in mycursor: print(x)

def get_energy_data(db,room_no,date_,time_):
    mycursor = db.cursor()
    mycursor.execute("SELECT energy_consumption FROM Energy WHERE room_no=%s AND date_=%s AND time_=%s",(room_no,date_,time_))
    for x in mycursor: return x

In [23]:
insert_schedule_data(db,'123','2024-03-04', '1200', '1')
insert_weather_data(db,'2024-03-04', '1200', 27.8)
insert_room_data(db,'123','2024-03-04', '1200',26.5,-1)
insert_energy_data(db,'123','2024-03-04', '1200', 3147)

(123, datetime.date(2024, 3, 4), 1200, 1)
(datetime.date(2024, 3, 4), 1200, 27.8)
(123, datetime.date(2024, 3, 4), 1200, 26.5, -1)
(123, datetime.date(2024, 3, 4), 1200, 3147.0)


In [24]:
import datetime
# Function to get all relevant data given a specific room and date_time slot

def get_data(room_no,date_,time_):
    date_obj = datetime.datetime.strptime(date_, '%Y-%m-%d')
    day_of_week = date_obj.strftime('%A')

    inside_temp = get_room_data(db,room_no,date_,time_)[0]
    outside_temp = get_weather_data(db,date_,time_)[0]
    booking_status = get_schedule_data(db,room_no,date_,time_)[0]
    energy_consumption = get_energy_data(db,room_no,date_,time_)[0]
    change_in_temp = get_room_data(db,room_no,date_,time_)[1]

    return [day_of_week,time_,inside_temp,outside_temp,booking_status,energy_consumption,change_in_temp]

print(get_data('123','2024-03-04', '1200'))

['Monday', '1200', 26.5, 27.8, 1, 3147.0, -1]
